# Workflows

Workflows are sequences of tasks that can be executed by a system, often without any human intervention or supervision. 

In this example, we'll build a recommendation engine workflow that can run unattended.

In [35]:
import os
import json
import getpass
from langchain_anthropic import ChatAnthropic

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")

model = ChatAnthropic(
    model="claude-3-7-sonnet-20250219",
    temperature=0,
    max_tokens=1024,
    max_retries=2,
    # other params...
)

In [24]:
import pandas as pd
from hashlib import md5
df = pd.read_csv("example data/financial-topics-csv.txt")
df["article_id"] = df["title"].map(lambda s: md5(str.encode(s)).hexdigest()[:8])
df.head()

,title,url,article_id
0,Building an Emergency Fund: Create Financial P...,NaN,813f295d
1,"Planning for retirement, your future self will...",https://www.notion.so/Planning-for-retirement-...,bfc16b4a
2,"Needs vs wants, how to prioritise your spending",https://www.notion.so/Needs-vs-wants-how-to-pr...,190d339d
3,How to achieve financial independence: The ste...,https://www.notion.so/How-to-achieve-financial...,86bc47d4
4,The power of compounding: How early investing ...,https://www.notion.so/The-power-of-compounding...,5595ecc5


In [25]:
articles_as_string = df.to_json(orient='records')  # List-like
articles_as_string

'[{"title":"Building an Emergency Fund: Create Financial Peace of Mind","url":null,"article_id":"813f295d"},{"title":"Planning for retirement, your future self will thank you","url":"https:\\/\\/www.notion.so\\/Planning-for-retirement-your-future-self-will-thank-you-1abd15602ba08019b780e8f19d2ab8b8?pvs=21","article_id":"bfc16b4a"},{"title":"Needs vs wants, how to prioritise your spending","url":"https:\\/\\/www.notion.so\\/Needs-vs-wants-how-to-prioritise-your-spending-1abd15602ba080afb981c79287c83756?pvs=21","article_id":"190d339d"},{"title":"How to achieve financial independence: The step-by-step guide","url":"https:\\/\\/www.notion.so\\/How-to-achieve-financial-independence-The-step-by-step-guide-1acd15602ba080debd17f52333e918ff?pvs=21","article_id":"86bc47d4"},{"title":"The power of compounding: How early investing pays off","url":"https:\\/\\/www.notion.so\\/The-power-of-compounding-How-early-investing-pays-off-1acd15602ba0803ba031d81c4508c8d3?pvs=21","article_id":"5595ecc5"},{"ti

In [26]:
user_profile="""**Personal Demographics and Life Stage**

- Age: 34
- Family: Married to Sarah (32, part-time graphic designer)
- Children: Emma (4) and Lucas (2)
- Living in: Ghent, Belgium
- Health: Good, active lifestyle
- Planning: Considering a larger home in next 3-5 years

**Financial Situation**

- Combined Income: €115,000/year (€85,000 from Thomas, €30,000 from Sarah)
- Savings: €45,000 in savings accounts
- Emergency Fund: €20,000
- Debt: €280,000 mortgage on current apartment
- Monthly Expenses: €3,800 (including €1,200 mortgage)
- Insurance: Basic health, home, and life insurance

**Employment Details**

- Senior Software Engineer at a scale-up
- 3 years at current company
- Existing benefits: Basic group insurance (Branch 21)
- Stock options: 0.1% equity (unvested)
- Potential promotion to Lead Engineer in next 12 months

**Investment Profile**

- Risk Tolerance: Moderate to high
- Investment Experience: Some experience with ETFs
- Current Investments: €15,000 in tech stocks and broad market ETFs
    - Alternative asset class: some Cryptocurrency
    - Alternative asset class: 2 oldtimer Porsches
- Time Horizon: 25-30 years for retirement
- Behavior: Checks investments weekly, tends to hold long-term

**Financial Goals**

- Retirement: Aims to retire by 60
- Target Retirement Income: €4,000/month
- Save for children's education (€100,000 each)
- Purchase larger home (€500,000 budget)
- Build investment portfolio for passive income

**Behavioral Factors**

- Financial Literacy: Above average due to tech background
- Spending Style: Pragmatic, research-oriented
- Decision Making: Analytical, data-driven
- Market Response: Generally stays calm during volatility
- Pain Points: Concerned about having too much in savings accounts

**Tax Considerations**

- Tax Bracket: Highest Belgian tax bracket
- Currently maximizing pension tax benefits
- Interested in tax-efficient investing strategies
- Wants to optimize salary package structure

**Existing Financial Products**

- Bank: KBC main account
- Investments: Bolero trading account
- Pension: Company group insurance + small personal pension saving
- Insurance: DKV healthcare, AG home insurance
- Property: 2-bedroom apartment in Ghent

**Key Concerns & Motivations:**

- Worried about saving enough for children's education
- Feels current pension planning is insufficient
- Interested in optimizing tax situation
- Wants to make savings work harder but unsure about best approach
- Looking for ways to benefit from tech industry growth
- Concerned about Belgian inflation and low interest rates
"""

system_prompt=f"""Your task is to make personalized article recommendations for relevant content for a specific user profile. 

<user_profile>
{user_profile}
<user_profile>

<articles>
{articles_as_string}
</articles>

Output the top 5 article id's in descending order. Output NO other text
Example output: [51, 41, 23, 51, 24]
Actual output: [
"""

print(system_prompt)

Your task is to make personalized article recommendations for relevant content for a specific user profile. 

<user_profile>
**Personal Demographics and Life Stage**

- Age: 34
- Family: Married to Sarah (32, part-time graphic designer)
- Children: Emma (4) and Lucas (2)
- Living in: Ghent, Belgium
- Health: Good, active lifestyle
- Planning: Considering a larger home in next 3-5 years

**Financial Situation**

- Combined Income: €115,000/year (€85,000 from Thomas, €30,000 from Sarah)
- Savings: €45,000 in savings accounts
- Emergency Fund: €20,000
- Debt: €280,000 mortgage on current apartment
- Monthly Expenses: €3,800 (including €1,200 mortgage)
- Insurance: Basic health, home, and life insurance

**Employment Details**

- Senior Software Engineer at a scale-up
- 3 years at current company
- Existing benefits: Basic group insurance (Branch 21)
- Stock options: 0.1% equity (unvested)
- Potential promotion to Lead Engineer in next 12 months

**Investment Profile**

- Risk Tolerance: M

In [27]:
response = model.invoke(system_prompt)

In [29]:
response.text()

'["f7340931", "c715d4f7", "bfc16b4a", "be715750", "72181843"]'

In [34]:
system_prompt=f"""Your task is to make personalized article recommendations for relevant content for a specific user profile. 

<user_profile>
{user_profile}
<user_profile>

<articles>
{articles_as_string}
</articles>

Output the top 5 article id's in descending order, with a match_score between 0 and 1. Output NO other text
Example output: [{{"article_id": "abc123", "match_score": 0.59}}]
Actual output: [
"""
response = model.invoke(system_prompt)
response.text()


'[{"article_id": "f7340931", "match_score": 0.95}, {"article_id": "c715d4f7", "match_score": 0.92}, {"article_id": "bfc16b4a", "match_score": 0.89}, {"article_id": "72181843", "match_score": 0.85}, {"article_id": "e57ec2ee", "match_score": 0.83}]'

In [37]:
recommendations = json.loads(response.text())
recommendations

[{'article_id': 'f7340931', 'match_score': 0.95},
 {'article_id': 'c715d4f7', 'match_score': 0.92},
 {'article_id': 'bfc16b4a', 'match_score': 0.89},
 {'article_id': '72181843', 'match_score': 0.85},
 {'article_id': 'e57ec2ee', 'match_score': 0.83}]